# Camera Capture

Here is a notebook for you to save images.
Change the folder name if you want, then run the following code. Below should be a widget showing the live camera feed and a button to save the image. When you save the image, text should appear that says "Success! Saved image_[number].jpg!".

In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Image, clear_output
import time
import threading
import os

# Initialize our camera
camera = Camera.instance()

Change the name of the folder you want these to be saved in here.

In [ ]:
# TODO: Change folder name HERE!!!
SAVE_DIR = "stopsign_images"

In [ ]:
# Setup Widgets
image = widgets.Image(format='jpeg', width=300, height=300)
save_button = widgets.Button(description='Save Image')
saved_count = 0
status_label = widgets.Label(value='Ready')
display(widgets.VBox([image, save_button, status_label]))

stop_event = threading.Event()

# Image Update Function
def update_image_loop():
    global ret
    while not stop_event.is_set():
        frame = camera.value
        image.value = bgr8_to_jpeg(frame)
        time.sleep(0.01)  # control the update rate

# Button Callback to Save Image
os.makedirs(SAVE_DIR, exist_ok=True)

def save_image_callback(b):
    global saved_count
    frame = camera.value
    filename = f"image_{saved_count}.jpg"
    filepath = os.path.join(SAVE_DIR, filename)
    with open(filepath, 'wb') as f:
        f.write(image.value)
    status_label.value = f"Success! Saved {filename}!"
    saved_count += 1

# Connect Callback
save_button.on_click(save_image_callback)

# Starts the Feed
camera.start()
thread = threading.Thread(target=update_image_loop, daemon=True)
thread.start()

In [ ]:
stop_event.set()
thread.join()
stop_event.clear()
camera.stop()